In [19]:
import polars as pl
import numpy as np
from datetime import date
import talib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()
plt.rcParams["figure.dpi"] = 300

In [48]:
np.random.seed(42)  # for reproducibility
open_ = np.random.normal(loc=100, scale=10, size=365) + np.linspace(0, 10, 365)
close = open_ + np.random.normal(loc=0, scale=2, size=365) + np.linspace(0, 10, 365)
high = [max(o, c) + np.random.uniform(0, 5) for o, c in zip(open_, close)]
low = [min(o, c) - np.random.uniform(0, 5) for o, c in zip(open_, close)]

df = pl.DataFrame(
    {
        "date": pl.date_range(date(2023, 1, 1), date(2023, 12, 31), eager=True),
        "open": open_,
        "close": close,
        "high": high,
        "low": low,
        "volume": np.random.randint(1_000, 10_000, 365),
    }
)

In [ ]:
def transform(df):
    for period in range(6, 28):
        rsi = talib.RSI(df["close"], timeperiod=period).rename(f"rsi_{period}")
        william = talib.WILLR(
            df["high"], df["low"], df["close"], timeperiod=period
        ).rename(f"william_{period}")
        mfi = talib.MFI(
            df["high"], df["low"], df["close"], df["volume"], timeperiod=period
        ).rename(f"mfi_{period}")
        macd = talib.MACD(df["close"], fastperiod=period, slowperiod=2 * period + 2)[
            0
        ].rename(f"macd_{period}")
        ppo = talib.PPO(
            df["close"], fastperiod=period, slowperiod=2 * period + 2
        ).rename(f"ppo_{period}")
        roc = talib.ROC(df["close"], timeperiod=period).rename(f"roc_{period}")
        # TODO: cmfi
        cmo = talib.CMO(df["close"], timeperiod=period).rename(f"cmo_{period}")
        sma = talib.SMA(df["close"], timeperiod=period).rename(f"sma_{period}")
        ema = talib.EMA(df["close"], timeperiod=period).rename(f"ema_{period}")
        wma = talib.WMA(df["close"], timeperiod=period).rename(f"wma_{period}")
        # TODO: hma
        tema = talib.TEMA(df["close"], timeperiod=period).rename(f"tema_{period}")
        cci = talib.CCI(df["high"], df["low"], df["close"], timeperiod=period).rename(
            f"cci_{period}"
        )
        # TODO: dpo
        # TODO: kst
        # TODO: eom
        # TODO: ibr
        dmi = talib.DX(df["high"], df["low"], df["close"], timeperiod=period).rename(
            f"dmi_{period}"
        )
        psar = talib.SAR(df["high"], df["low"]).rename(f"psar_{period}")

        df = df.with_columns(
            rsi, william, mfi, macd, ppo, roc, cmo, sma, ema, wma, tema, cci, dmi, psar
        )
    return df

In [63]:
transform(df)

date,open,close,high,low,volume,rsi_6,william_6,mfi_6,macd_6,ppo_6,rsi_7,william_7,mfi_7,macd_7,ppo_7,rsi_8,william_8,mfi_8,macd_8,ppo_8,rsi_9,william_9,mfi_9,macd_9,ppo_9,rsi_10,william_10,mfi_10,macd_10,ppo_10,rsi_11,william_11,mfi_11,macd_11,ppo_11,rsi_12,…,macd_20,ppo_20,rsi_21,william_21,mfi_21,macd_21,ppo_21,rsi_22,william_22,mfi_22,macd_22,ppo_22,rsi_23,william_23,mfi_23,macd_23,ppo_23,rsi_24,william_24,mfi_24,macd_24,ppo_24,rsi_25,william_25,mfi_25,macd_25,ppo_25,rsi_26,william_26,mfi_26,macd_26,ppo_26,rsi_27,william_27,mfi_27,macd_27,ppo_27
date,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-01-01,104.967142,104.164701,105.059095,99.483639,5107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-02,98.64483,99.120487,103.691981,98.384973,1560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-03,106.53183,106.61196,107.200716,103.825349,9785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04,115.312716,115.590486,118.473068,111.767414,6548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05,97.768356,96.332227,99.138632,91.977381,2826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-12-27,115.083575,124.605718,125.443131,112.505178,1631,57.066608,-12.031292,29.948127,0.671472,-1.348816,56.140303,-12.031292,43.120184,0.61719,-0.500844,55.428947,-12.031292,39.950842,0.577955,-0.119005,54.859488,-12.031292,31.723719,0.540243,0.553181,54.390501,-29.222836,36.326179,0.499922,2.45325,53.996589,-29.222836,47.184428,0.456992,2.823919,53.661,…,0.178798,-0.092111,52.014063,-25.295856,61.854232,0.171539,-0.323144,51.913971,-25.295856,58.112247,0.168892,-0.711045,51.823048,-25.295856,61.511705,0.170522,-1.031121,51.740185,-25.295856,62.406776,0.176076,-1.445606,51.664437,-25.295856,59.270446,0.185202,-1.412425,51.594994,-25.295856,55.565012,0.197554,-1.364546,51.531159,-25.295856,57.521816,0.21281,-1.157376
2023-12-28,125.244972,135.199422,135.722261,120.648012,1905,65.473361,-1.407945,44.816649,3.057897,0.291151,63.673932,-1.348781,35.591017,2.714171,0.725095,62.242863,-1.348781,46.897705,2.447107,0.594175,61.070674,-1.348781,43.663852,2.226986,0.504072,60.08992,-1.348781,35.119289,2.037916,1.534626,59.256286,-2.102902,39.289326,1.87148,2.871102,58.538965,…,1.005268,0.178212,54.882944,-1.820313,61.174916,0.96093,0.203888,54.649104,-1.820313,62.609306,0.924163,-0.034176,54.434889,-1.820313,58.892164,0.894287,-0.608631,54.238061,-1.820313,62.171165,0.870655,-0.854264,54.056708,-1.820313,63.036171,0.85